# Using the cache

First make sure that the dependencies are installed:

```bash
pip install -r requirements-dev.txt
```

Then run redis using docker compose:

```bash
docker compose -f docker-compose.redis.yml up -d
```

Then run the notebook. 

If you want to view the data in redis, you can use the redis-cli:

```bash
brew install redis
redis-cli -p 6380
```

In [1]:
import fsspec  
from redis_fsspec_cache.asyn import RedisAsyncCachingFilesystem

fs = fsspec.filesystem(
    "s3",
    anon=True,
)

cached_fs = RedisAsyncCachingFilesystem(
    redis_host="localhost",
    redis_port=6380,
    expiry_time=60,
    fs=fs,
)

Let try to access a single chunk from a remote GRIB file we know about in an s3 bucket:

```json
"gust\/0.0.0": [
    "s3:\/\/noaa-hrrr-bdp-pds\/hrrr.20230927\/conus\/hrrr.t00z.wrfsubhf00.grib2",
    2884409,
    1133540
],
```

In [2]:
url = "s3://noaa-hrrr-bdp-pds/hrrr.20230927/conus/hrrr.t00z.wrfsubhf00.grib2"
offset = 2884409
length = 1133540
end = offset + length

In [3]:
%%time

nocache_chunk = fs.cat_file(url, start=offset, end=end)

CPU times: user 112 ms, sys: 85.2 ms, total: 197 ms
Wall time: 650 ms


In [4]:
%%time

uncached_chunk = cached_fs.cat_file(url, start=offset, end=end)

CPU times: user 13.2 ms, sys: 5.38 ms, total: 18.5 ms
Wall time: 141 ms


In [5]:
%%time

cached_chunk = cached_fs.cat_file(url, start=offset, end=end)

CPU times: user 12.3 ms, sys: 4.98 ms, total: 17.3 ms
Wall time: 170 ms


In [7]:
is_correct_length = len(cached_chunk) == length
is_correct_data = cached_chunk == uncached_chunk and cached_chunk == nocache_chunk
print(f"Correct length: {is_correct_length}")
print(f"Correct data: {is_correct_data}")

Correct length: True
Correct data: True
